In [1]:
import numpy as np
#import jax.numpy as jnp

# imports custom RNCRN functions that are used throughout multiple  notebooks 
import os, sys
from pathlib import Path

module_path = str(Path(os.path.abspath('')).parent)
if module_path not in sys.path:
    sys.path.append(module_path)

from functions.target_systems import parametric_disjoint_cycle
from functions.RNCRN_tools import save_mat_model_basic_rncrn
from functions.RNCRN_train import train_basic_RNCRN, initialize_single_RNCRN
from functions.create_attractors_tools import draw_attractor_parametric
from functions.RNCRN_phase_plane import merge_multi_state_data


In [2]:
# Invokes the code for Algorithm 1 (to generate vector field)
# sets the parameter \delta
distance = 0.01

# sets the parameter \eta
magnitude = 1

# sets the parameter K (number of padding points)
number_of_padding_each_side = 20

# sets the interval of the variable that is used in the parametric equation to create a line of points 
low_parametric = 0
upper_parametric = 2*np.pi
step_parametric = 0.01
parametric_concentric_inner = lambda t: parametric_disjoint_cycle(t, 0.5, 4.5, 1.5)
parametric_concentric_outer = lambda t: parametric_disjoint_cycle(t, 0.5, 1.5, 1.5)

# calls a function that implements Algorithm 1 - each circle
inputs_a, targets_a = draw_attractor_parametric(parametric_concentric_inner, low_parametric, upper_parametric, step_parametric, distance, magnitude, number_of_padding_each_side)
inputs_b, targets_b = draw_attractor_parametric(parametric_concentric_outer, low_parametric, upper_parametric, step_parametric, distance, magnitude, number_of_padding_each_side)
inputs, targets = merge_multi_state_data([inputs_a, inputs_b], [targets_a, targets_b])


In [3]:
# sets the hyper-structure of the RNCRN (i.e. N and M)   
number_of_exec_species = 2
number_of_chemical_perceptrons = 15

In [4]:
# calls a function that randomly initializes the RNCRN for training with automatic differentiation  
rnd_seed = 2
params = initialize_single_RNCRN(number_of_exec_species, number_of_chemical_perceptrons, rnd_seed)

In [5]:
start_learning_rate = 5
batch_size = 200
error_threshold = 0.3
number_of_epochs = 400

loss, params, lowest_params, lowest_loss = train_basic_RNCRN(inputs, targets, params, number_of_epochs, start_learning_rate, batch_size, error_threshold)

Epoch: 200 loss =  8.916009
Lowest loss found!


In [6]:
save_mat_model_basic_rncrn(params, number_of_chemical_perceptrons, number_of_exec_species, loss, rnd_seed)